In [14]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver import Keys, ActionChains
import time, os

profile = 'Default'#Jaque
profile = 'Profile 2'#Vó
profile = 'Profile 7'#Mineiro

options = webdriver.ChromeOptions()
options.add_argument("user-data-dir=C:/Users/Felipe/AppData/Local/Google/Chrome/User Data")
options.add_argument(f'profile-directory={profile}')
##options.add_argument('headless')

driver = webdriver.Chrome(options=options)



In [1]:
import time
print(time.strftime('%H'))

15


In [52]:
import sqlite3
from pathlib import Path

ROOT_DIR = Path('').parent
DB_NAME = 'db.sqlite3'
DB_FILE = ROOT_DIR / DB_NAME
connection = sqlite3.connect(DB_FILE)
cursor = connection.cursor()

for page in range(1,11):
    driver.get(f'https://www.mercadolivre.com.br/vendas/omni/lista?startPeriod=WITH_DATE_CLOSED_6M_OLD&sort=DATE_CLOSED_DESC&page={page}')

    vendas_lista_html = \
        driver.find_elements(By.CSS_SELECTOR, '#desktop > div.sc-page > div > div.sc-tab-content > div > div.sc-list.sc-list-marketplace > div.andes-card')
    

    for venda_html in vendas_lista_html:
        produto  = compra_html.find_element(By.TAG_NAME, 'a').text
        vendedor = compra_html.find_element(By.CSS_SELECTOR, 'div.list-item > div.text-with-link > span > span').text
        codigo   = compra_html.find_element(By.CSS_SELECTOR, 
                                            'div.list-item > div.bf-ui-button-container.bf-ui-button-container--vertical.bf-ui-button-container--transparent')\
                                            .find_element(By.TAG_NAME, 'a').get_attribute('href')
        status = compra_html.find_element(By.CSS_SELECTOR,'div.list-item > div.list-item__product > div > p.list-item__intro > span').text.lower()
        
        if status == 'entregue':
            cursor.execute(
                f'SELECT COUNT(id) FROM compras WHERE numero = "{codigo}"'
            )
            result = cursor.fetchone()[0]
            if not result > 0:
                comando_sql = f'INSERT INTO compras (produto, vendedor, numero) VALUES ("{produto}","{vendedor}","{codigo}")'
                cursor.execute(
                    comando_sql
                )
        
connection.commit()
cursor.close()
connection.close()


Funções de envio de mensagem e shit_enter(para quebrar linha dentro do input de msg)

In [3]:
def shift_enter(driver):
        ActionChains(driver)\
            .key_down(Keys.SHIFT)\
            .send_keys(Keys.ENTER)\
            .key_up(Keys.SHIFT)\
            .perform()
    

def envia_msg(nome, telefone, msg, enviar=True):
    driver.get(f'https://web.whatsapp.com/send?phone={telefone}')
    
    input_box_msg = WebDriverWait.__class__
    # Esperar para encontrar o input
    input_box_msg = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located(
            (By.CSS_SELECTOR, '#main > footer > div._2lSWV._3cjY2.copyable-area > div > span:nth-child(2) > div > div._1VZX7 > div._3Uu1_ > div.g0rxnol2.ln8gz9je.lexical-rich-text-input > div.to2l77zo.gfz4du6o.ag5g9lrv.bze30y65.kao4egtt > p')
        )
    )

    input_box_msg.send_keys(f'Olá {nome}')
    #shift_enter(driver)
    if type(msg) == str:
        input_box_msg.send_keys(msg)
    if type(msg) == set:
        for m in msg:
            input_box_msg.send_keys(m[0]+' - '+m[1])
            shift_enter(driver)
            shift_enter(driver)
    if enviar:
        #Enter para enviar msg
        input_box_msg.send_keys(Keys.ENTER)

    time.sleep(1)

In [4]:
import sqlite3
from pathlib import Path
from selenium.common.exceptions import TimeoutException

ROOT_DIR = Path('').parent
DB_NAME = 'db.sqlite3'
DB_FILE = ROOT_DIR / DB_NAME

connection = sqlite3.connect(DB_FILE)
cursor = connection.cursor()
log_erro = set()
#ler para quem enviar e números
cursor.execute(f'SELECT id, nome, telefone FROM envios')

for row in cursor.fetchall():
    cod, nome, telefone = row
    try:
        compras = set()
        cursor.execute('SELECT produto, vendedor FROM compras')
        for compra in cursor.fetchall():
            produto, vendedor = compra
            compras.add((produto, vendedor,))
        envia_msg(nome, telefone, compras, False)
    except TimeoutException as err:
        cursor.execute(f'UPDATE envios SET log = "Tempo excedido ou telefone inválido" WHERE id = {cod}')

connection.commit()

cursor.close()
connection.close()

AttributeError: 'WebElement' object has no attribute 'key_down'

In [49]:
#anexar arquivo

#Clicka no botão +
driver.find_element(By.CSS_SELECTOR, '#main > footer > div._2lSWV._3cjY2.copyable-area > div > span:nth-child(2) > div > div._2xy_p._1bAtO > div > div > div > div > span').click()
time.sleep(1)
#Acha o input de arq
input_file = driver.find_element(By.CSS_SELECTOR, '#main > footer > div._2lSWV._3cjY2.copyable-area > div > span:nth-child(2) > div > div._2xy_p._1bAtO > div > div > span > div > ul > div > div:nth-child(1) > li > div > input[type=file]')
s = os.path.abspath('envios.xlsx') +' '+ os.path.abspath('envios2.xlsx')
print(s)
input_file.send_keys(s)
time.sleep(5)

"""
#Acha o input de arq 2
input_file2 = driver.find_element(By.CSS_SELECTOR, '#app > div > div > div._2QgSC > div._2Ts6i._2xAQV > span > div > span > div > div > div.g0rxnol2.thghmljt.p357zi0d.rjo8vgbg.ggj6brxn.f8m0rgwh.gfz4du6o.r7fjleex.bs7a17vp.ov67bkzj > div > input[type=file]')
input_file2.send_keys(os.path.abspath('envios2.xlsx'))
time.sleep(5)

#Acha o input de arq 3
input_file3 = driver.find_element(By.CSS_SELECTOR, '#app > div > div > div._2QgSC > div._2Ts6i._2xAQV > span > div > span > div > div > div.g0rxnol2.thghmljt.p357zi0d.rjo8vgbg.ggj6brxn.f8m0rgwh.gfz4du6o.r7fjleex.bs7a17vp.ov67bkzj > div > input[type=file]')
input_file3.send_keys(os.path.abspath('envios.xlsx'))
time.sleep(5)
"""

#driver.find_element(By.CSS_SELECTOR, '#app > div > div > div._2QgSC > div._2Ts6i._2xAQV > span > div > span > div > div > div.g0rxnol2.thghmljt.p357zi0d.rjo8vgbg.ggj6brxn.f8m0rgwh.gfz4du6o.r7fjleex.bs7a17vp.ov67bkzj > div > div.O2_ew > div._3wFFT > div > div > span').click()


c:\Users\gusta\Projects\wpp_v1\envios.xlsx c:\Users\gusta\Projects\wpp_v1\envios2.xlsx


InvalidArgumentException: Message: invalid argument: File not found : c:\Users\gusta\Projects\wpp_v1\envios.xlsx c:\Users\gusta\Projects\wpp_v1\envios2.xlsx
  (Session info: chrome=116.0.5845.140)
Stacktrace:
	GetHandleVerifier [0x00007FF7AF8F52A2+57122]
	(No symbol) [0x00007FF7AF86EA92]
	(No symbol) [0x00007FF7AF73E3AB]
	(No symbol) [0x00007FF7AF771CE5]
	(No symbol) [0x00007FF7AF796FDA]
	(No symbol) [0x00007FF7AF76EB76]
	(No symbol) [0x00007FF7AF7971F0]
	(No symbol) [0x00007FF7AF7AEB82]
	(No symbol) [0x00007FF7AF796DB3]
	(No symbol) [0x00007FF7AF76D2B1]
	(No symbol) [0x00007FF7AF76E494]
	GetHandleVerifier [0x00007FF7AFB9EF82+2849794]
	GetHandleVerifier [0x00007FF7AFBF1D24+3189156]
	GetHandleVerifier [0x00007FF7AFBEACAF+3160367]
	GetHandleVerifier [0x00007FF7AF986D06+653702]
	(No symbol) [0x00007FF7AF87A208]
	(No symbol) [0x00007FF7AF8762C4]
	(No symbol) [0x00007FF7AF8763F6]
	(No symbol) [0x00007FF7AF8667A3]
	BaseThreadInitThunk [0x00007FFE99C726AD+29]
	RtlUserThreadStart [0x00007FFE9B

In [50]:
import sqlite3
from pathlib import Path
from selenium.common.exceptions import TimeoutException

ROOT_DIR = Path('').parent
DB_NAME = 'db.sqlite3'
DB_FILE = ROOT_DIR / DB_NAME

connection = sqlite3.connect(DB_FILE)
cursor = connection.cursor()

cursor.execute(f'DELETE FROM COMPRAS')
cursor.execute(f'DELETE FROM ENVIOS')
cursor.execute(f'INSERT INTO ENVIOS (NOME, TELEFONE) VALUES ("Cris","5547992912406") ')
cursor.execute(f'INSERT INTO ENVIOS (NOME, TELEFONE) VALUES ("Erro","55479632992") ')
cursor.execute(f'INSERT INTO ENVIOS (NOME, TELEFONE) VALUES ("Gustavo","554796329923") ')


connection.commit()

cursor.close()
connection.close()